In [1]:
save_plots = True
train_model = True
save_model = True
flops_count = True

In [3]:
import os
import numpy as np
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import seaborn as sns
from sklearn.datasets import make_moons
from sklearn.neighbors import KernelDensity

In [6]:
print(f"Current directory: {os.getcwd()}")
X = np.load('data/X.npy')
y = np.load('data/z.npy')
n_samples, n_class = X.shape

data = torch.tensor(X, dtype=torch.float32)
print(f"observed data size {data.shape[0]}")

Current directory: /Users/sohamdas/Downloads/two_moon_data
observed data size 100000


In [5]:
HIDDEN_DIM = 32
LATENT_DIM = 2
LEARNING_RATE = 0.001
NUM_EPOCHS = 50
BATCH_SIZE = 128
GENERATED_SAMPLE = 1000
KL_WEIGHT = 0.1

PRINT_FREQ = 10
PRINT_LAP = max(1, NUM_EPOCHS/PRINT_FREQ)

In [6]:
from VAE_moon_model import VAE_moon_unlab, train, model_save, model_load, count_flops

2024-11-21 22:04:33.072122: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 22:04:34.106282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
model = VAE_moon_unlab(HIDDEN_DIM, LATENT_DIM).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
dataset = TensorDataset(data)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
print(f'Using {device} device')

Using cpu device


In [8]:
if flops_count:
    flops, params = count_flops(model)
    print('Computational complexity: ', flops)
    print('Number of parameters: ', params)

VAE_moon_unlab(
  4.61 k, 100.000% Params, 4.61 KMac, 100.000% MACs, 
  (fc1): Linear(96, 2.081% Params, 96.0 Mac, 2.081% MACs, in_features=2, out_features=32, bias=True)
  (fc2): Linear(1.06 k, 22.887% Params, 2.11 KMac, 45.774% MACs, in_features=32, out_features=32, bias=True)
  (fc3): Linear(1.06 k, 22.887% Params, 0.0 Mac, 0.000% MACs, in_features=32, out_features=32, bias=True)
  (fc31): Linear(66, 1.430% Params, 66.0 Mac, 1.430% MACs, in_features=32, out_features=2, bias=True)
  (fc32): Linear(66, 1.430% Params, 66.0 Mac, 1.430% MACs, in_features=32, out_features=2, bias=True)
  (fc4): Linear(96, 2.081% Params, 96.0 Mac, 2.081% MACs, in_features=2, out_features=32, bias=True)
  (fc5): Linear(1.06 k, 22.887% Params, 1.06 KMac, 22.887% MACs, in_features=32, out_features=32, bias=True)
  (fc6): Linear(1.06 k, 22.887% Params, 1.06 KMac, 22.887% MACs, in_features=32, out_features=32, bias=True)
  (fc7): Linear(66, 1.430% Params, 66.0 Mac, 1.430% MACs, in_features=32, out_features=2, b

In [9]:
if train_model:
    start_time = time.time()
    model = train(model, dataloader, optimizer, device, NUM_EPOCHS, PRINT_LAP, KL_WEIGHT)
    end_time = time.time()
    print(f"Total time for {NUM_EPOCHS} epochs {(end_time-start_time)/60:.2f} mins")    

Epoch [1/50] -> loss:51.47
Epoch [6/50] -> loss:34.67
Epoch [11/50] -> loss:34.49
Epoch [16/50] -> loss:34.48
Epoch [21/50] -> loss:34.46
Epoch [26/50] -> loss:34.39
Epoch [31/50] -> loss:34.34
Epoch [36/50] -> loss:34.32
Epoch [41/50] -> loss:34.34
Epoch [46/50] -> loss:34.41
Total time for 50 epochs 1.86 mins


In [10]:
if train_model and save_model:
    model_save(model, file_path=root_loc+"/vae_moon_model.pth")

Model saved to /home/sohamdas/Python/two_moon_data/vae_moon_model.pth


In [11]:
if not train_model:
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    model_class = VAE_moon_unlab(HIDDEN_DIM, LATENT_DIM).to(device)
    model = model_load(model_class, "vae_moon_model.pth", device=device)

In [12]:
model.eval()
with torch.no_grad():
    z = torch.randn(GENERATED_SAMPLE, LATENT_DIM).to(device)
    generated_data = model.decode(z).cpu().numpy()

In [13]:
from VAE_moon_plot import plot_temp, plot_contours, plot_generated_data
from VAE_moon_plot import get_latent_representation, plot_latent_parameters, plot_latent_variable

In [14]:
mu, logvar = get_latent_representation(model, data, device)

In [15]:
print("Working on the plots ...")
plot_temp(generated_data, save_plots)
plot_contours(data, save_plots)
plot_generated_data(data,generated_data, save_plots)
plot_latent_parameters(mu, logvar, save_plots, labels=y)
plot_latent_variable(mu, logvar, save_plots, labels=y)

Working on the plots ...
